## INST414 | Assignment 4 - Module 4 | Daniel Hernandez Gonzalez

In [1]:
# Pandas, we all love them. 
import pandas as pd

#spotify tools.
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Tools for making calculations
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Authenticating with Spotify API

In [2]:
# Authentication keys, taken out from my final push.
client_id = '85b32a4fe99f44db9d9665c3281bd8a5'
client_secret = '8598677237e74986ac35810334a418d3'
username = 'ks65h2anf54k24x4n4jvfx2hy'

# set up Spotify API credentials
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Gathering data

#### Setting up API call

In [12]:
# Ver. 3.0 from what I started with this part of code. 
# It is more efficient to make a function to do this set of instructions instead of writing it more than once. 
def get_songs(genre, year):
    # set of 50 songs per API request
    query = f'genre:{genre}, year:{year}'
    limit = 50
    offset = 0
    top_songs = []

    # Spotify API calls for each set of 50 songs
    for i in range(4):
        results = sp.search(q=query, type='track', limit=limit, offset=offset)
        top_songs += results['tracks']['items']
        offset += limit

    return top_songs[:200]


In [13]:
# Checking output until I 200 total items
testing = get_songs('rap', '2022')
print(len(testing))

200


#### Gathering top 200 songs from various years for the following genres. 

In [15]:
# I chose specific genres based on the genres shown on Spotify's platform and a few that I am interested in. Genre page: https://open.spotify.com/genre/hub-browse-grid
genre_list = ['rap', 'pop', 'house', 'country', 'rock', 'soul', 'inde', 'punk', 'corrido', 'banda']

# These are the years I will be assessing. There is no particular reason why I chose to go back in increments of two. I think more distance could yield better insights. 
years = [2022, 2020, 2018, 2016, 2014]

# Creating an empty DataFrame with columns.
songs_df = pd.DataFrame(columns=['track_id', 'track_name', 'artist_name', 'year', 'genre', 'tempo', 'popularity', 'acousticness', 'danceability', 'energy', 'loudness'])


In [16]:
# Iterating over the years and genres to add data into songs_df
for year in years:
    for genre in genre_list:
        genre_songs = get_songs(genre, year)
        for track in genre_songs:
            track_id = track['id']
            artist_names = [artist['name'] for artist in track['album']['artists']]
            artist_name = ', '.join(artist_names)
            track_name = track['name']
            popularity = track['popularity']
            genre = genre

            # Spotify API call to get additional audio features
            audio_features = sp.audio_features(track_id)
            if audio_features:
                acousticness = audio_features[0]['acousticness']
                danceability = audio_features[0]['danceability']
                energy = audio_features[0]['energy']
                tempo = audio_features[0]['tempo']
                loudness = audio_features[0]['loudness']
            else:
                # Incase any missing values. Will drop after.
                acousticness, danceability, energy, tempo = None, None, None, None

            # adding all data to pulled into songs_df
            songs_df.loc[len(songs_df)] = [track_id, track_name, artist_name, year, genre, tempo, popularity, acousticness, danceability, energy, loudness]
            

In [19]:
# Show current dataframe
songs_df

,track_id,track_name,artist_name,year,genre,tempo,popularity,acousticness,danceability,energy,loudness
0,4FyesJzVpA39hbYvcseO2d,Just Wanna Rock,Lil Uzi Vert,2022,rap,150.187,91,0.0652,0.486,0.545,-7.924
1,1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,2022,rap,88.980,94,0.0521,0.644,0.735,-5.747
2,2dHHgzDwk4BJdRwy9uXhTO,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,2022,rap,97.950,96,0.4170,0.715,0.620,-6.005
3,1bDbXMyjaUIooNwFE9wn0N,Rich Flex,"Drake, 21 Savage",2022,rap,153.150,91,0.0503,0.561,0.520,-9.342
4,0vjeOZ3Ft5jvAi9SBFJm1j,Superhero (Heroes & Villains) [with Future & C...,Metro Boomin,2022,rap,116.622,90,0.1520,0.526,0.606,-5.300
...,...,...,...,...,...,...,...,...,...,...,...
9995,24mxFLqaipbeGVyQrmWL7Q,El Triste Alegre,Calibre 50,2014,banda,176.791,46,0.2830,0.602,0.888,-2.206
9996,0l69McY2BZSq8GLJ8faaJ8,Matar o Morir,Noel Torres,2014,banda,120.057,37,0.4080,0.777,0.846,-2.820
9997,6getVY7VDYzTukR60IwXFm,Ni Hablar,El Trono de Mexico,2014,banda,98.063,38,0.1460,0.619,0.717,-4.555
9998,7Jj94cHHF6cdpPyhP7SxVC,El Criminal,"Lorenzo De Monteclaro, Cornelio Reyna",2014,banda,95.213,40,0.4050,0.738,0.504,-6.756


In [20]:
# Dataframe size before cleaning
print(f'table size before cleaning: {songs_df.shape}') 

# Dropping duplicate track names if they have the same artist name
songs_df = songs_df.drop_duplicates(subset=['track_name', 'artist_name'])

# Dataframe size after cleaning and displaying table
print(f'\ntable size after cleaning: {songs_df.shape}\n') 
songs_df

table size before cleaning: (10000, 11)

table size after cleaning: (8793, 11)



,track_id,track_name,artist_name,year,genre,tempo,popularity,acousticness,danceability,energy,loudness
0,4FyesJzVpA39hbYvcseO2d,Just Wanna Rock,Lil Uzi Vert,2022,rap,150.187,91,0.0652,0.486,0.545,-7.924
1,1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,2022,rap,88.980,94,0.0521,0.644,0.735,-5.747
2,2dHHgzDwk4BJdRwy9uXhTO,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,2022,rap,97.950,96,0.4170,0.715,0.620,-6.005
3,1bDbXMyjaUIooNwFE9wn0N,Rich Flex,"Drake, 21 Savage",2022,rap,153.150,91,0.0503,0.561,0.520,-9.342
4,0vjeOZ3Ft5jvAi9SBFJm1j,Superhero (Heroes & Villains) [with Future & C...,Metro Boomin,2022,rap,116.622,90,0.1520,0.526,0.606,-5.300
...,...,...,...,...,...,...,...,...,...,...,...
9993,0TuNVZPZqrxwGdh5S0fTow,Máxima Potencia,Banda Tierra Sagrada,2014,banda,112.493,46,0.5150,0.800,0.722,-3.934
9994,69hsR7V6aiSjecIiYqxlNb,Aprovechate,El Trono de Mexico,2014,banda,76.035,39,0.2960,0.710,0.656,-4.160
9997,6getVY7VDYzTukR60IwXFm,Ni Hablar,El Trono de Mexico,2014,banda,98.063,38,0.1460,0.619,0.717,-4.555
9998,7Jj94cHHF6cdpPyhP7SxVC,El Criminal,"Lorenzo De Monteclaro, Cornelio Reyna",2014,banda,95.213,40,0.4050,0.738,0.504,-6.756


In [94]:
# Drops a row if it is missing a value.


print(f"{songs_df.shape}\n")

print(songs_df.isna().sum())
#songs_df.dropna(inplace=True)

#print(f"\n{songs_df.shape}")

# I was getting errors in pervious versions of my code. I kept this incase I wanted to come back and test different genres. 

(1323, 10)

track_id        0
track_name      0
artist_name     0
genre           0
tempo           0
popularity      0
acousticness    0
danceability    0
energy          0
loudness        0
dtype: int64


In [96]:
# pulling the top songs from each genre and sorting by popularity for better view.
top_songs_by_genre = songs_df.sort_values('popularity', ascending=False).groupby('genre').head(1)
top_songs_by_genre

,track_id,track_name,artist_name,genre,tempo,popularity,acousticness,danceability,energy,loudness
2,2dHHgzDwk4BJdRwy9uXhTO,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,rap,97.950,96,0.417000,0.715,0.620,-6.005
153,0WtM2NBVQNNJLh6scP13H8,Calm Down (with Selena Gomez),"Rema, Selena Gomez",pop,106.999,96,0.382000,0.801,0.806,-5.206
1200,1lRtH4FszTrwwlK5gTSbXO,AMG,"Natanael Cano, Peso Pluma, Gabito Ballesteros",corrido,136.175,95,0.152000,0.772,0.730,-6.657
308,4zN21mbAuaD0WqtmaTZZeP,Ferrari,"James Hype, Miggy Dela Rosa",house,125.004,86,0.012700,0.847,0.690,-7.877
1351,5kpxVMDvYGRmNqawPqDTYR,JGL,"La Adictiva, Luis R Conriquez",banda,112.852,84,0.550000,0.702,0.586,-5.317
612,3CIyK1V4JEJkg02E4EJnDl,Enemy (with JID) - from the series Arcane Leag...,Imagine Dragons,rock,77.011,83,0.237000,0.728,0.783,-4.424
452,48UKTR66uUOT9LaUvooTNx,The Kind of Love We Make,Luke Combs,country,102.025,82,0.022600,0.608,0.745,-4.130
760,48CKzGWOC65n8a8kdANJV8,Only Love Can Hurt Like This - Slowed Down Ver...,"Paloma Faith, sped up + slowed",soul,82.012,74,0.102000,0.554,0.854,-6.263
1054,2ah5gOCogw00A62XBoepmc,Miracle,A Day To Remember,punk,160.028,68,0.000266,0.463,0.917,-3.776
900,21rpBPAlXAvBavvu5kCQzs,After Midnight,Phoenix,inde,100.010,61,0.002450,0.618,0.811,-6.654


#### As a disclaimer, I am aware that this is alphabetically the top song per genre based on sorting by popularity. 

# Distance calculations

### My goal: 
    - Grab the top songs from each genre. 
    - Calculate Euclidean distance between songs.
    - print top 10 closest songs

In [97]:
def closest_tracks(track_id, songs_df):
    # Extracting audio features for the provided track
    audio_features = songs_df.loc[songs_df['track_id'] == track_id, ['tempo', 'acousticness', 'danceability', 'energy', 'loudness']]

    # Calculating Euclidean distance between the provided track and all other tracks
    songs_df.loc[:,'distance'] = np.sqrt((songs_df['tempo'] - audio_features['tempo'].iloc[0])**2 +
                                    (songs_df['danceability'] - audio_features['danceability'].iloc[0])**2 +
                                    (songs_df['acousticness'] - audio_features['acousticness'].iloc[0])**2 +
                                    (songs_df['energy'] - audio_features['energy'].iloc[0])**2 + 
                                    (songs_df['loudness'] - audio_features['loudness'].iloc[0])**2)

    # Sorting the DataFrame by distance in ascending order
    songs_df = songs_df.sort_values('distance')

    # Adding the input track back to the top of the DataFrame
    input_track = songs_df.loc[songs_df['track_id'] == track_id]

    # Adding closest tracks to new dataframe
    closest_tracks = pd.concat([input_track, songs_df.loc[((songs_df['track_name'] != input_track['track_name'].iloc[0]) |
                                        (songs_df['artist_name'] != input_track['artist_name'].iloc[0])) &
                                        (songs_df['track_id'] != input_track['track_id'].iloc[0])].head(10)])

    # Resetting the index and dropping old index to avoid too many numbers. 
    closest_tracks = closest_tracks.reset_index(drop=True)

    return closest_tracks

### What are the closest songs in comparison to the top rap song?

In [98]:
rap_id = '2dHHgzDwk4BJdRwy9uXhTO' # "'Creepin' (with The Weeknd & 21 Savage)"
closest_rap = closest_tracks(rap_id, songs_df)
closest_rap

,track_id,track_name,artist_name,genre,tempo,popularity,acousticness,danceability,energy,loudness,distance
0,2dHHgzDwk4BJdRwy9uXhTO,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,rap,97.950,96,0.41700,0.715,0.620,-6.005,0.000000
1,4ZUpTMKPYFTfJqamWFU9Zv,IDGAF (with blackbear),BoyWithUke,rap,97.975,76,0.42700,0.782,0.728,-5.930,0.150010
2,4uk8GguFehn7djGvFJTUUT,Enloquecido,Grupo Arriesgado,corrido,98.053,71,0.25000,0.784,0.734,-6.007,0.237190
3,5Ei8EK2uXNkqqVSe2QYsFp,Que Te Vaya Bien,Julión Álvarez y su Norteño Banda,banda,97.990,76,0.23700,0.667,0.467,-5.906,0.263655
4,3bvJftZKZe5QKz433NczyV,No Es Por Acá,Carin Leon,corrido,97.568,82,0.45600,0.746,0.370,-6.309,0.550747
5,6Awl7JBDQjfm7xkS3LkhDi,We Got History,Mitchell Tenpenny,country,98.025,74,0.00140,0.628,0.833,-6.327,0.578766
6,1VMBapbhHidf6ALFszT7w1,Lonely,Imagine Dragons,rock,97.983,73,0.02560,0.714,0.730,-5.564,0.600720
7,3G1nE9ZEv4eGEM48IAIv8u,Millennium,Toro y Moi,soul,98.055,51,0.32500,0.687,0.691,-6.743,0.754956
8,1zsPaEkglFvxjAhrM8yhpr,Hey Mor,Ozuna,pop,98.002,92,0.00302,0.901,0.589,-6.713,0.843151
9,6Rpbzk2LC8xWZI93RyodAq,Hey Mor,"Ozuna, Feid",pop,98.002,84,0.00302,0.901,0.589,-6.713,0.843151


### What are the closest songs in comparison to the top corrido song?

In [99]:
corrido_id = '1lRtH4FszTrwwlK5gTSbXO' # "AMG"
closest_corrido = closest_tracks(corrido_id, songs_df)
closest_corrido

,track_id,track_name,artist_name,genre,tempo,popularity,acousticness,danceability,energy,loudness,distance
0,1lRtH4FszTrwwlK5gTSbXO,AMG,"Natanael Cano, Peso Pluma, Gabito Ballesteros",corrido,136.175,95,0.1520,0.772,0.730,-6.657,0.000000
1,3gfSH9aYZbyGjdmbeoWliw,Siempre Pendientes,"Peso Pluma, Luis R Conriquez",corrido,135.929,87,0.3310,0.774,0.748,-7.086,0.526238
2,5gVCfYmQRPy1QJifP8f5gg,Fall In Love,Bailey Zimmerman,pop,135.962,80,0.4510,0.524,0.643,-6.055,0.752494
3,1wLNEMiUzwvRZz9XHCXhAE,Fifth of May,Zach Bryan,country,136.728,72,0.3910,0.596,0.544,-7.039,0.757909
4,1ENsFKcyksWFyXNQlWCI1c,Mirate Nomas (En Vivo),Ulices Chaidez,banda,135.836,55,0.5820,0.653,0.375,-7.060,0.776155
5,6AhwAWzSlISc5ZvGonkgdN,Playing God,Polyphia,rock,137.060,67,0.0395,0.601,0.789,-6.822,0.925110
6,5VfkfKTTo5BRKqHwfENF0V,Míranos Ahora,Calibre 50,corrido,135.832,70,0.6510,0.680,0.439,-5.909,1.009603
7,46FdwGMcgz6gik4RCpWBAW,Los Collares,"Oscar Maydon, El Padrinito Toys",corrido,136.724,86,0.2080,0.645,0.712,-5.787,1.038215
8,45bfH0GZvUyujIBiKRhXso,B.O.T.A. (Baddest Of Them All),"Eliza Rose, Interplanetary Criminal",house,136.981,67,0.0671,0.756,0.962,-7.526,1.210820
9,39JofJHEtg8I4fSyo7Imft,B.O.T.A. (Baddest Of Them All) - Edit,"Eliza Rose, Interplanetary Criminal",house,137.001,82,0.1640,0.736,0.906,-7.589,1.258299


### What are the closest songs in comparison to the top Banda song?

In [100]:
banda_id = '5kpxVMDvYGRmNqawPqDTYR' # "JGL"
closest_banda = closest_tracks(banda_id, songs_df)
closest_banda

,track_id,track_name,artist_name,genre,tempo,popularity,acousticness,danceability,energy,loudness,distance
0,5kpxVMDvYGRmNqawPqDTYR,JGL,"La Adictiva, Luis R Conriquez",banda,112.852,84,0.550000,0.702,0.586,-5.317,0.000000
1,0RstZCli3DcyY1LJq38P4b,El Chaman,Gabito Ballesteros,corrido,112.998,69,0.260000,0.814,0.575,-5.348,0.345025
2,42cVZAhX13JCHwrvlfqLcB,When You Lose Someone,The Dip,soul,112.452,53,0.365000,0.751,0.498,-5.766,0.637159
3,1xTYy3K3hiTcoZfoD7culn,Say You Want Me - Single Version,Masego,soul,112.978,63,0.190000,0.884,0.658,-5.940,0.756249
4,6DL44bNYvHzr1E58BAx5zx,Horseshit on Route 66,The Garden,punk,112.328,49,0.028300,0.498,0.978,-5.115,0.884777
5,66MvNxKQGPQS0AUeaoYlcn,hair out,Stand Atlantic,punk,112.987,53,0.000488,0.678,0.896,-4.685,0.903487
6,5Tg3TbTftDrjmQORfsqw5m,Enamorado Y Feliz,La Adictiva,banda,113.590,50,0.377000,0.653,0.434,-5.895,0.966521
7,5DAkzBJ48N7z6lwY4eZ0PP,Artefact,Phoenix,inde,112.025,53,0.007840,0.705,0.636,-5.734,1.074367
8,11kskoMyNDumaR30CI6rRv,Half Of Me,Thomas Rhett,country,111.995,69,0.006830,0.575,0.842,-5.058,1.085462
9,69w5X6uTrOaWM32IetSzvO,Daydreaming,Harry Styles,pop,113.971,77,0.311000,0.707,0.811,-5.298,1.166316
